# What could go wrong and how to fix it

## 2. Take a step back: look at the full Bayesian workflow
<img src="janfb/figures/what-if-i-told-you-this-is-not-fully-bayesian.jpg" align="center" alt="beadexample" width="500"/>

TBD.